Bài 1

In [12]:
import sqlite3
import math

conn = sqlite3.connect('data.db')
conn.create_function('SQRT', 1, math.sqrt)
conn.create_function('POWER', 2, lambda x, y: x**y)
cursor = conn.cursor()

# Tạo bảng và chèn dữ liệu như trên
cursor.execute('''
CREATE TABLE IF NOT EXISTS data_table (
    id INTEGER PRIMARY KEY,
    A REAL,
    B REAL
)
''')
conn.commit()

cursor.execute("DELETE FROM data_table")  # Xóa dữ liệu cũ
sample_data = [
    (1, 10, 20),
    (2, 15, 25),
    (3, 20, 30),
    (4, 25, 35),
    (5, 30, 40)
]
cursor.executemany('INSERT INTO data_table (id, A, B) VALUES (?, ?, ?)', sample_data)
conn.commit()

# Tính hệ số tương quan bằng SQL
query = '''
SELECT 
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) /
    SQRT(
        (COUNT(*) * SUM(A * A) - POWER(SUM(A), 2))
        *
        (COUNT(*) * SUM(B * B) - POWER(SUM(B), 2))
    ) AS correlation
FROM data_table
'''
cursor.execute(query)
result = cursor.fetchone()
print("Hệ số tương quan Pearson:", result[0])

conn.close()

Hệ số tương quan Pearson: 1.0


Bài 2

In [16]:
import sqlite3
import math
import pandas as pd
from scipy.stats import chi2_contingency

# Bước 1: Tạo database, bảng và chèn dữ liệu

# Kết nối đến SQLite (tạo file data.db nếu chưa có)
conn = sqlite3.connect('data.db')

# Đăng ký các hàm toán học cho SQLite (SQRT và POWER)
conn.create_function('SQRT', 1, math.sqrt)
conn.create_function('POWER', 2, lambda x, y: x**y)

cursor = conn.cursor()

# Xóa bảng nếu đã tồn tại và tạo bảng mới
cursor.execute("DROP TABLE IF EXISTS car_scores;")
cursor.execute("""
CREATE TABLE car_scores (
    day   VARCHAR(10),
    model VARCHAR(1),
    score FLOAT
);
""")
conn.commit()

# Xóa dữ liệu cũ (nếu có) và chèn dữ liệu mẫu
cursor.execute("DELETE FROM car_scores;")
sample_data = [
    ('Day1', 'A', 8.0),
    ('Day1', 'B', 9.0),
    ('Day1', 'C', 7.0),
    ('Day2', 'A', 7.5),
    ('Day2', 'B', 8.0),
    ('Day2', 'C', 7.5),
    ('Day3', 'A', 8.0),
    ('Day3', 'B', 9.0),
    ('Day3', 'C', 7.0),
    ('Day4', 'A', 8.0),
    ('Day4', 'B', 9.0),
    ('Day4', 'C', 7.0)
]
cursor.executemany("INSERT INTO car_scores (day, model, score) VALUES (?, ?, ?)", sample_data)
conn.commit()

# Bước 2: Pivot dữ liệu theo day để kiểm tra phân phối các nhóm điểm
# Phân loại điểm (score) thành 3 nhóm:
# - Low: score < 7
# - Medium: 7 ≤ score < 9
# - High: score ≥ 9
query_pivot = """
SELECT 
    day,
    SUM(CASE WHEN score < 7 THEN 1 ELSE 0 END) AS Low,
    SUM(CASE WHEN score >= 7 AND score < 9 THEN 1 ELSE 0 END) AS Medium,
    SUM(CASE WHEN score >= 9 THEN 1 ELSE 0 END) AS High
FROM car_scores
GROUP BY day;
"""
cursor.execute(query_pivot)
print("Bảng tần suất theo ngày (Day × [Low, Medium, High]):")
for row in cursor.fetchall():
    print(row)

# Bước 3: Đọc dữ liệu qua Pandas để tạo bảng tần suất theo model
df = pd.read_sql_query("SELECT * FROM car_scores", conn)

# Đóng kết nối sau khi lấy xong dữ liệu
conn.close()

# Hàm phân loại điểm
def categorize_score(s):
    if s < 7:
        return 'Low'
    elif s < 9:
        return 'Medium'
    else:
        return 'High'

# Tạo cột mới: score_category
df['score_category'] = df['score'].apply(categorize_score)

print("\nDữ liệu sau khi thêm cột phân loại (score_category):")
print(df)

# Tạo bảng tần suất (contingency table) theo mẫu xe (model) và score_category
contingency_table = pd.crosstab(df['model'], df['score_category'])
print("\nBảng tần suất (Contingency Table):")
print(contingency_table)

# Bước 4: Thực hiện kiểm định Chi-square
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print("\nKết quả kiểm định Chi-square:")
print("Chi2 statistic =", chi2)
print("p-value        =", p_value)
print("Degrees of Freedom =", dof)
print("Expected frequencies:\n", expected)


Bảng tần suất theo ngày (Day × [Low, Medium, High]):
('Day1', 0, 2, 1)
('Day2', 0, 3, 0)
('Day3', 0, 2, 1)
('Day4', 0, 2, 1)

Dữ liệu sau khi thêm cột phân loại (score_category):
     day model  score score_category
0   Day1     A    8.0         Medium
1   Day1     B    9.0           High
2   Day1     C    7.0         Medium
3   Day2     A    7.5         Medium
4   Day2     B    8.0         Medium
5   Day2     C    7.5         Medium
6   Day3     A    8.0         Medium
7   Day3     B    9.0           High
8   Day3     C    7.0         Medium
9   Day4     A    8.0         Medium
10  Day4     B    9.0           High
11  Day4     C    7.0         Medium

Bảng tần suất (Contingency Table):
score_category  High  Medium
model                       
A                  0       4
B                  3       1
C                  0       4

Kết quả kiểm định Chi-square:
Chi2 statistic = 7.999999999999999
p-value        = 0.018315638888734186
Degrees of Freedom = 2
Expected frequencies:
 [[1. 3.]


KẾT LUẬN :

-Kết quả thử nghiệm phụ thuộc vào mẫu xe

-Không phụ thuộc vào ngày

BÀI 3

In [21]:
import sqlite3

# 1. Kết nối cơ sở dữ liệu SQLite
conn = sqlite3.connect("flights.db")
cursor = conn.cursor()

# 2. Tạo bảng flights và chèn dữ liệu mẫu
cursor.execute("DROP TABLE IF EXISTS flights")  # Xóa nếu đã có bảng cũ
cursor.execute("""
CREATE TABLE flights (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    departure_time INTEGER
)
""")

# Thêm dữ liệu mẫu 
sample_times = [(830,), (1445,), (50,), (1230,), (2102,)]
cursor.executemany("INSERT INTO flights (departure_time) VALUES (?)", sample_times)
conn.commit()

# 3. Hàm Python để chuyển số nguyên sang định dạng thời gian
def format_time(time_int):
    time_str = f"{time_int:04d}"  # Đảm bảo 4 chữ số (0030, 0830, etc.)
    hour = int(time_str[:2])
    minute = time_str[2:]

    period = "AM" if hour < 12 else "PM"
    hour_12 = hour % 12
    hour_12 = 12 if hour_12 == 0 else hour_12

    return f"{hour_12}:{minute} {period}"

# 4. Thêm cột mới formatted_time vào bảng để lưu thời gian chuyển đổi
cursor.execute("ALTER TABLE flights ADD COLUMN formatted_time TEXT")

# 5. Cập nhật các giá trị formatted_time
cursor.execute("SELECT id, departure_time FROM flights")
rows = cursor.fetchall()
for row in rows:
    flight_id, dep_time = row
    formatted = format_time(dep_time)
    cursor.execute("UPDATE flights SET formatted_time = ? WHERE id = ?", (formatted, flight_id))
conn.commit()

# 6. Hiển thị kết quả cuối cùng
print("Dữ liệu sau khi chuyển đổi:")
for row in cursor.execute("SELECT * FROM flights"):
    print(row)

# 7. Đóng kết nối
conn.close()


Dữ liệu sau khi chuyển đổi:
(1, 830, '8:30 AM')
(2, 1445, '2:45 PM')
(3, 50, '12:50 AM')
(4, 1230, '12:30 PM')
(5, 2102, '9:02 PM')


BÀI 4

In [22]:
import sqlite3
import statistics

# Kết nối SQLite
conn = sqlite3.connect("doanh_thu.db")
cursor = conn.cursor()

# Tạo bảng và thêm dữ liệu mẫu
cursor.execute("""
CREATE TABLE IF NOT EXISTS doanh_thu (
    so_tien REAL
)
""")
cursor.execute("DELETE FROM doanh_thu")  # Xóa dữ liệu cũ nếu có
sample_data = [(200,), (210,), (190,), (205,), (1000,), (195,), (215,), (220,), (198,)]
cursor.executemany("INSERT INTO doanh_thu (so_tien) VALUES (?)", sample_data)
conn.commit()


PHÂN TÍCH NGOẠI LỆ (Z-score)
- Trung bình: 292.56
- Độ lệch chuẩn: 265.47
- Ngưỡng Z-score: ±2.5

CÁC GIÁ TRỊ NGOẠI LỆ:
- 1000.00 (Z-score = 2.66)


BÀI 5

In [24]:
import sqlite3
import pandas as pd

# Kết nối đến database
conn = sqlite3.connect("company.db")
cursor = conn.cursor()

# Tạo bảng Employee và thêm dữ liệu mẫu
cursor.execute("DROP TABLE IF EXISTS Employee")
cursor.execute("""
    CREATE TABLE Employee (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        last_name TEXT,
        salary REAL,
        age INTEGER
    )
""")

# Thêm dữ liệu mẫu (giả lập có thể trùng nhau)
sample_employees = [
    ("Nguyen", 1200, 30),
    ("Nguyen", 1180, 29),
    ("Le", 1500, 40),
    ("Le", 2400, 42),
    ("Tran", 1350, 33),
    ("Tran", 1345, 34),
    ("Pham", 1800, 28)
]

cursor.executemany("INSERT INTO Employee (last_name, salary, age) VALUES (?, ?, ?)", sample_employees)
conn.commit()

# Truy vấn phát hiện các nhân viên có họ giống và lương gần giống (chênh không quá 1000)
query = """
SELECT 
    e1.last_name AS name1,
    e2.last_name AS name2,
    e1.salary AS salary1,
    e2.salary AS salary2,
    ABS(e1.salary - e2.salary) AS salary_diff,
    e1.age AS age1,
    e2.age AS age2
FROM Employee e1
JOIN Employee e2
  ON e1.id < e2.id
WHERE e1.last_name = e2.last_name
  AND ABS(e1.salary - e2.salary) <= 1000;
"""

# Thực hiện truy vấn và hiển thị kết quả
df = pd.read_sql_query(query, conn)
print("📋 Các cặp nhân viên có thể là cùng người hoặc bị nhập trùng:\n")
print(df)

# Đóng kết nối
conn.close()



📋 Các cặp nhân viên có thể là cùng người hoặc bị nhập trùng:

    name1   name2  salary1  salary2  salary_diff  age1  age2
0  Nguyen  Nguyen   1200.0   1180.0         20.0    30    29
1      Le      Le   1500.0   2400.0        900.0    40    42
2    Tran    Tran   1350.0   1345.0          5.0    33    34
